# Creating a character level language model to generate new dinosaurus names.

In [1]:
# Requirements
import numpy as np
from rnn_lstm_blocks import *
import random
import copy

In [2]:
# Loading data
data = open('data/dinos.txt', 'r').read()
data= data.lower()
chars = sorted(list(set(data)))
data_size, vocab_size = len(data), len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [3]:
# Functions


def initialize_parameters(n_a, n_x, n_y):
    """
    Initialize parameters with small random values
    
    Returns:
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    """
    np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters


def update_parameters(parameters, gradients, lr):
    """
    Updates the RNN parameters using gradient descent.

    Arguments:
    parameters -- Dictionary containing the RNN parameters:
                  Wax -- Weight matrix for input to hidden state, shape (n_a, n_x)
                  Waa -- Weight matrix for hidden to hidden state, shape (n_a, n_a)
                  Wya -- Weight matrix for hidden to output, shape (n_y, n_a)
                  b   -- Bias vector for hidden state, shape (n_a, 1)
                  by  -- Bias vector for output, shape (n_y, 1)
    
    gradients -- Dictionary containing the gradients of the loss with respect to the parameters:
                 dWax -- Gradient of Wax, shape (n_a, n_x)
                 dWaa -- Gradient of Waa, shape (n_a, n_a)
                 dWya -- Gradient of Wya, shape (n_y, n_a)
                 db   -- Gradient of b, shape (n_a, 1)
                 dby  -- Gradient of by, shape (n_y, 1)

    lr -- Learning rate (a positive float), determines the step size in gradient descent

    Returns:
    parameters -- Updated dictionary of parameters after applying gradient descent
    """
    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    
    return parameters


def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    loss, cache = rnn_forward_character_generation(X, Y, a_prev, parameters)
    gradients, a = rnn_backward_character_generation(X, Y, parameters, cache)
    gradients = clip(gradients, 5)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X)-1]


def get_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    return txt

In [4]:
# Training and generating 
def model(data_x, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27, verbose = False):
    """
    Trains the model and generates dinosaur names. 
    
    Arguments:
    data_x -- text corpus, divided in words
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text (size of the vocabulary)
    
    Returns:
    parameters -- learned parameters
    """
    
    # Initializing
    n_x, n_y = vocab_size, vocab_size
    parameters = initialize_parameters(n_a, n_x, n_y)
    loss = -np.log(1.0/vocab_size)*dino_names
    examples = [x.strip() for x in data_x]
    np.random.seed(0)
    np.random.shuffle(examples)
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        idx = j % len(examples) # since num_iterations > examplles, idx wraps around to the beginning when it exceeds the number of examples
        single_example = examples[idx]
        single_example_chars = [c for c in single_example]
        single_example_ix = [char_to_ix[c] for c in single_example_chars]
        X = [None] + single_example_ix
        ix_newline = char_to_ix['\n']
        Y = X[1:] + [char_to_ix['\n']]
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
        loss = loss * 0.999 + curr_loss * 0.001 # to keep the loss smooth.
        
        # debug statements to aid in correctly forming X, Y
        if verbose and j in [0, len(examples) -1, len(examples)]:
            print("j = " , j, "idx = ", idx,) 
        if verbose and j in [0]:
            print("single_example =", single_example)
            print("single_example_chars", single_example_chars)
            print("single_example_ix", single_example_ix)
            print(" X = ", X, "\n", "Y =       ", Y, "\n")

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            for name in range(dino_names):
                sampled_indices = sample(parameters, char_to_ix)
                last_dino_name = get_sample(sampled_indices, ix_to_char)
                print(last_dino_name.replace('\n', ''))
        
    return parameters, last_dino_name

parameters, last_name = model(data.split("\n"), ix_to_char, char_to_ix, 22001, verbose = True)

j =  0 idx =  0
single_example = turiasaurus
single_example_chars ['t', 'u', 'r', 'i', 'a', 's', 'a', 'u', 'r', 'u', 's']
single_example_ix [20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19]
 X =  [None, 20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19] 
 Y =        [20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19, 0] 

Iteration: 0, Loss: 23.087336

Co
Eyhqgyshubjerbjgmis
Qfguyhvbqtdkxb
Okuxhjmozrjgycsxqkeisqywvxgijorg
Jhywwqinubuavuapmdctozzzcyflteoebci
Kqjbagdzduoomziuojypckurxgtloyzwiwqk
Tmutuhunlr
j =  1535 idx =  1535
j =  1536 idx =  0
Iteration: 2000, Loss: 27.884160

Amycgangwsacrusycripsiurgtorandprusayrabhustactgny
Honqmeleprusamglonggpodanrosennnsteroshuceranelocr
Inlonaleonxyorytonganytabtonosaunus
Padeleloypurapfnorusaurus
Aurus
Itreriptixtisgurros
Ontepokosauluspmurinontyaicallpeosaurus
Iteration: 4000, Loss: 25.901815

Amkorosaurus
Stdoosaurus
Rusaurus
Eschuszuratanstipanos
Chmmoop
Alorosaurus
Azarosaurus
Iteration: 6000, Loss: 24.608779

Crlhux
Raptorfirus
Deliosaurus
Hua
Fhidosaurushs
S
Eheosa